<a href="https://colab.research.google.com/github/danyentezari/bn-ocr/blob/main/code_inverted_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install lxml stemming

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stemming: filename=stemming-1.0.1-py3-none-any.whl size=11138 sha256=c2d6093913a0a9311ed6b0ef2f996f8a26c7d101a5322b2a2d49cf15122dd120
  Stored in directory: /root/.cache/pip/wheels/6b/e5/e2/c52ebc0a5b53fd82b00cc385e57bb1c90bd50e5f54ddbc06d1
Successfully built stemming


In [3]:
!git clone https://github.com/danyentezari/trec500.git
!cp -r './trec500/CW1collection' './CW1collection'

Cloning into 'trec500'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [9]:
# Note:
# To run this script and generate the results files (index.txt, results.boolean.txt, 
# and resukts.ranked.txt), set the path to the XML collection file here. The code 
# for generating the output files are in sections (6), (7), and (8).
xmlCollectionPath = './CW1collection/trec.5000.xml'
print(xmlCollectionPath)

# ------------------------------------------------------------------------------------------ #
# (1) Import Libraries and Collection
# ------------------------------------------------------------------------------------------ #

# Import the libraries
import math
import re
from stemming.porter2 import stem

# List of stop words
from stop_words import stopWords

# Read the collection
from lxml import etree
import xml.etree.ElementTree as ET
root = ET.parse( xmlCollectionPath ).getroot()



# ------------------------------------------------------------------------------------------ #
# (2) Reading the Documents, Creating the Inverted Index 
# ------------------------------------------------------------------------------------------ #
def posIndex(tokens, invertedIndex, docID):
    """
    Function to add to the inverted index the position 
    of terms of a document.
    """
    for tokenIndex, token in enumerate(tokens):
        # If token has already been seen
        if token in invertedIndex.keys():
            # Increment the general frequency of the token
            invertedIndex[token][0] += 1

            # Add position of new encountered term

            # If docID is already in the dictionary of the token,
            # add the next token index
            if docID in invertedIndex[token][1].keys():
                invertedIndex[token][1][docID].append(tokenIndex)
            # Otherwise, add new docID and token index of current
            # token encounter
            else:
                invertedIndex[token][1][docID] = []
                invertedIndex[token][1][docID].append(tokenIndex)
        # Otherwise, add token to dictionary
        else:   
            # Initiate vector (list) for token
            invertedIndex[token] = []
            # Increment general frequency by 1
            invertedIndex[token].append(1)
            # Append to vector positional vector (including pos of first token encounter)
            invertedIndex[token].append({ docID: [tokenIndex]})
    return invertedIndex


def preprocessor(document):
    """
    Function to preprocess documents and return lists where
    the tokens have been made lowercase, stripped of non-letter
    characters, and stemmed.
    """
    # Case Folding
    documentLowerCase = document.casefold()

    # Remove punctuation
    # documentLowerCase = documentLowerCase.search(r'(\.|\:|\;|\,|\!|\?|\"|\'|\[|\]|\{|\})', documentLowerCase)
    documentNoPunct = re.sub(r'([^\w]|\d|\s)', ' ', documentLowerCase)

    # Tokenize
    tokens = re.split(r'\s+', documentNoPunct)
    tokens1 = tokens[1:]

    # Tokens without stop words
    # Tokens with stemming
    tokensWithoutStopWords = []

    for token in tokens1:
        if token not in stopWords:
            tokensWithoutStopWords.append(stem(token))

    return tokensWithoutStopWords


# Put the documents into 'documents' list
# for indexing
documents = []
for elem in root.findall('DOC'):
    headlineContent = elem.find('HEADLINE').text
    textContent = elem.find('TEXT').text
    docno = elem.find('DOCNO').text
    documents.append( 
        {
            "docID": docno,
            "text": headlineContent + " " + textContent,
            "tokens": preprocessor( headlineContent + " " + textContent )
        } 
    )

# Build the invertedIndex
invertedIndex = {}
for document in documents:
    # invertedIndex = termCountAndIndex(tokens, invertedIndex, docIndex)
    invertedIndex = posIndex(document['tokens'], invertedIndex, document['docID'])



# ------------------------------------------------------------------------------------------ #
# (3) Intersect and Poitional Intersect Functions
# ------------------------------------------------------------------------------------------ #
def nextPointer(pointer, index, plist):
    """
    Function to return the next pointer for a posting
    """
    if index+1 > len(plist)-1:
        pointer = None
    else:
        index += 1
        pointer = plist[int(index)]
    return pointer, index


def getDocuments(posting):
    """
    Returns the documents IDs of a posting as a 
    list data type.
    """
    return list(posting[1].keys())


def getPositions(posting):
    """
    Returns the term positions (indices) for a
    posting as a list data type.
    """
    return list(posting[1].values())


def intersect(posting1, posting2):
    """
    Function to intersect two postings. This is
    the implementation of the intersect algorithm. 
    """
    answer = []
    p1docs = getDocuments(posting1)
    p2docs = getDocuments(posting2)


    i = 0
    j = 0

    docPointer1 = p1docs[i]
    docPointer2 = p2docs[j]

    while docPointer1 != None and docPointer2 != None:
        if int(docPointer1) == int(docPointer2):
            answer.append(docPointer1)
           
            docPointer1, i = nextPointer(docPointer1, i, p1docs)
            docPointer2, j = nextPointer(docPointer2, j, p2docs)

        elif int(docPointer1) < int(docPointer2):
            docPointer1, i = nextPointer(docPointer1, i, p1docs)
        else:
            docPointer2, j = nextPointer(docPointer2, j, p2docs)
    return answer


def posIntersect(posting1, posting2, k):
    """
    Function to intersect two postings in k proximity. This is
    the implementation of the positional intersect algorithm.  
    """
    # List containing documents where match is found.
    # List is tuple (but List implementations) with
    # values (docID, posOfWTerm1, posOfWTerm2)
    answer = []

    # Indices for enumerating document IDs
    i = j = 0

    # List of documents for each term
    post1_ids = getDocuments(posting1)
    post2_ids = getDocuments(posting2)

    # docID for each document at current index
    docID1 = post1_ids[i]
    docID2 = post2_ids[j]

    while docID1 != None and docID2 != None:
        if int(docID1) == int(docID2):
            l = []

            # Positions of terms in current documents
            docPos1 = getPositions(posting1)[i]
            docPos2 = getPositions(posting2)[j]        
           
            # Indices for positions of terms
            ii = jj = 0

            # Point to...
            while ii != len(docPos1):
                while jj != len(docPos2):
                    if abs( docPos1[ii] - docPos2[jj]) <= k:
                        l.append(docPos2[jj])
                    elif docPos2[jj] > docPos1[ii]:
                        break
                    jj += 1

                while l != [] and abs(l[0] - docPos1[ii]) > k:
                    del l[0]
                
                for ps in l:
                    answer.append( [docID1, docPos1[ii], ps] )
                ii += 1

            docID1, i = nextPointer(docID1, i, post1_ids)
            docID2, j = nextPointer(docID2, j, post2_ids)

        elif int(docID1) < int(docID2):
            docID1, i = nextPointer(docID1, i, post1_ids)
        else:
            docID2, j = nextPointer(docID2, j, post2_ids)
    return answer



# ------------------------------------------------------------------------------------------ #
# (4) TF-IDF Functions
# ------------------------------------------------------------------------------------------ #

def idf(df, N):
    """
    Calculates and returns the inverse-document frequency
    for a given document frequency (df) and size of documents
    in a collection (N).
    """
    return math.log10(N/df)


def tf(term,docID,invertedIndex):
    """
    Calculates and returns the term frequency in the collection
    for a given term.
    """
    if docID not in list(invertedIndex[term][1].keys()):
        return 0
    else:
        return len(invertedIndex[term][1][docID])


def get_df(term, invertedIndex):
    """
    Returns the document frequency of a term as a list.
    """
    return len(invertedIndex[term][1])


def tf_idf_normal(term, docID, invertedIndex, documents):
    """
    Calculates TF-IDF (term frequency - inverse document frequency)
    and applies normalization priort to return the score. 
    """
    collectionSize = len(documents)
    df = get_df(term, invertedIndex)
    idf_val = idf(df, collectionSize)
    tf_val = tf(term, docID, invertedIndex)
    if tf_val == 0:
        return 0
    else:
        tf_idf = (1 + math.log10(tf_val) ) * idf_val
        return tf_idf


def score(terms, docID, invertedIndex, documents):
    """
    Returns the ranking score after aggregating the normalized
    tf-idf for each search term.
    """
    s = 0
    for term in terms:
        s += tf_idf_normal(term, docID, invertedIndex, documents)
    return ( docID, round(s,4) )



# ------------------------------------------------------------------------------------------ #
# (5) Search Functions
# ------------------------------------------------------------------------------------------ #

def phraseSearch(queryString, k=1):
    """
    Function to search for phrase queries. Unless the k is specified,
    the default proximity is 1. Queries must begin with double quotes ("").
    """
    phraseQueries = re.finditer(r'"(\w|\d|\s{0,})+"', queryString)
    invertedIndexKeys = list(invertedIndex.keys())
    result = []

    for pq in phraseQueries:
        phraseToken = pq.group().replace("\"", "")
        # print( phraseToken )
        tokens = re.split(r'\s+', phraseToken)
        preprocessed_tl = []
        for i, t in enumerate(tokens):
            tl = t.casefold()
            tl = stem(tl)
            preprocessed_tl.append(tl)

            if i > 0:
                if preprocessed_tl[i - 1] in invertedIndexKeys and preprocessed_tl[i] in invertedIndexKeys:
                    result += posIntersect(
                            invertedIndex[ preprocessed_tl[i-1] ], 
                            invertedIndex[ preprocessed_tl[i] ],
                            k
                        )
    return result


def booleanSearch(queryString):
    """
    Function to search for terms containing the logical
    operators 'AND', 'OR', and 'NOT' using the set operations
    intersection, union, and complement, respectively.
    """
    queryTokens = re.split("\s+", queryString)
    querySet = set({})
    searchTerms = []

    for i, t in enumerate(queryTokens):
        if t[0] == '"':

            # Remove '+' for intermediate processing
            term = t.replace("+"," ")
            matches = phraseSearch(term)
            docs = set({})
            for m in matches:
                docs.add(m[0])

            searchTerms.append(term)
            uSet = docs

            if i>=0 and queryTokens[i-1] == 'OR':
                querySet = querySet.union( uSet )

            elif i>=0 and queryTokens[i-1] == 'AND':

                querySet = querySet.intersection( uSet )
            elif i>=0 and queryTokens[i-1] == 'NOT':
                querySet = querySet - uSet

            else:
                querySet = querySet.union( uSet )
        
        else:
            if t != 'AND' and t != 'OR' and t!= 'NOT' and t != '' and t != ' ':
                # Processed term
                pTerm = stem(t.strip().casefold()) 
                uSet = set(invertedIndex[pTerm][1].keys())
                searchTerms.append(pTerm)

                if i>=0 and queryTokens[i-1] == 'OR':
                    querySet = querySet.union( uSet )

                elif i>=0 and queryTokens[i-1] == 'AND':
                    querySet = querySet.intersection( uSet )

                elif i>=0 and queryTokens[i-1] == 'NOT':
                    querySet = querySet - uSet

                else:
                    querySet = querySet.union( uSet )

    return searchTerms, querySet


def positionalSearch(queryString):
    """
    This function will search for terms using the positional
    intersect function (posIntersect) for queries of the form 
    #k(term1, term2)
    """
    positionSearch = re.search('#[0-9]+', queryString).group()

    if bool(positionSearch):
        maxDistance = int(positionSearch[1:])
        positionalTerms = re.split('(#[0-9]+)', queryString)

        queryTokens = re.split('(#[0-9]+|\(|\)|\,)', queryString)
        positionalTerms = []
        for t_i, t in enumerate(queryTokens):
            
            if t_i > 0:
                if t != '' and t != ',' and t != '(' and t != ')' and ('#' not in t):
                    t = t.replace(' ', '')
                    t = stem(t.strip().casefold())
                    positionalTerms.append(t)

    t1 = invertedIndex[ positionalTerms[0] ]
    t2 = invertedIndex[ positionalTerms[1] ]
    return posIntersect(t1, t2, maxDistance)


def processQuery(queryString):
    """
    Function to process a query containing phrases and logical 
    operators for the boolean retrieval.
    """
    phraseQueries = re.finditer(r'"(\w|\d|\s{0,})+"', queryString)
    searchTokens = []
    for q in phraseQueries: 
        s,e = q.span()

        qlist = list(queryString)
        for c in range(s,e):
            if qlist[c] == ' ':
                qlist[c] = '+'
            queryString = "".join(qlist)
    return queryString


def rankedRetrieval(queryString, invertedIndex, documents):
    """
    Function to process a query containing phrases and logical 
    operators for phrase search.
    """
    results = phraseSearch(queryString)
    matchingDocs = []
    for r in results:
        matchingDocs.append(r[0])

    searchTerms = queryString.split(" ")

    results = []
    for docID in list(matchingDocs):
        results.append( score(searchTerms, docID, invertedIndex,documents ) )

    results = sorted(results, key = lambda i: i[1], reverse=True)
    return results


def rankedRetrieval_freeform(queryString, invertedIndex, documents):
    """
    Use boolean search to search for free form queries. 
    This function will return the normalized tf-idf for all
    matches in order of score (descending).
    """
    searchTerms, matchingDocs = booleanSearch(queryString)

    results = []
    for docID in list(matchingDocs):
        results.append( score(searchTerms, docID, invertedIndex,documents ) )

    results = sorted(results, key = lambda i: i[1], reverse=True)
    return results


def booleanRetrieval(queryString):
    """
    Checks query to determine whether to use positional search
    or boolean search based. Will return return values of from
    either positionalSearch() or booleanSearch() depending on 
    the type of string argument.
    """
    positionSearch = re.search('#[0-9]+', queryString)
    if bool(positionSearch):
        return positionalSearch(queryString)
    else:
        queryString = processQuery(queryString)
        return booleanSearch(queryString)



print(invertedIndex)

./CW1collection/trec.5000.xml


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
list(invertedIndex.values())[0:10]

[[5512,
  {'5000': [0],
   '5001': [0],
   '5002': [0],
   '5003': [0],
   '5004': [0],
   '5005': [0],
   '5006': [0],
   '5007': [0],
   '5008': [0],
   '5009': [0],
   '5010': [0],
   '5011': [0],
   '5012': [0],
   '5013': [0],
   '5014': [0],
   '5015': [0],
   '5016': [0],
   '5017': [0],
   '5018': [0],
   '5019': [0],
   '5020': [0],
   '5021': [0],
   '5022': [0],
   '5023': [0],
   '5024': [0],
   '5025': [0],
   '5026': [0],
   '5027': [0],
   '5028': [0],
   '5029': [0],
   '5030': [0],
   '5031': [0],
   '5032': [0],
   '5033': [0],
   '5034': [0],
   '5035': [0],
   '5036': [0],
   '5037': [0],
   '5038': [0],
   '5039': [0],
   '5040': [0],
   '5041': [0],
   '5042': [0],
   '5043': [0],
   '5044': [0],
   '5045': [0],
   '5046': [0],
   '5047': [0],
   '5048': [0],
   '5049': [0],
   '5050': [0],
   '5051': [0],
   '5052': [0],
   '5053': [0],
   '5054': [0],
   '5055': [0],
   '5056': [0],
   '5057': [0],
   '5058': [0],
   '5059': [0],
   '5060': [0],
   '5061': [0],
